In [1]:
import sys
sys.path.append("D:\\WETrak")

import os
import os.path as path
import numpy as np
import pandas as pd
import json
import torch.utils.data
from glob import glob

import import_ipynb
import branch_adapter.regression.regression_data_utils as data_utils
import branch_adapter.classification.classification_cascade_inference as classification_cascade_inference
 
os.environ['KMP_DUPLICATE_LIB_OK']='True'

importing Jupyter notebook from D:\WETrak\branch_adapter\regression\regression_data_utils.ipynb
importing Jupyter notebook from D:\WETrak\branch_adapter\classification\classification_cascade_inference.ipynb
importing Jupyter notebook from D:\WETrak\branch_adapter\classification\classification_network.ipynb
importing Jupyter notebook from D:\WETrak\branch_adapter\classification\classification_data.ipynb
importing Jupyter notebook from D:\WETrak\branch_adapter\classification\classification_data_utils.ipynb
importing Jupyter notebook from D:\WETrak\branch_adapter\classification\classification_model_utils.ipynb


In [2]:
#[0,default]按照classify_num划分
class regression_data(torch.utils.data.Dataset):
    # output: 
    # emg: [batch,1,1000,2] min,max: [batch, 8]
    def __init__(self, root_path, user_name, split, finger_type_list, classify_num, output_path = None, inference_mode=False):
        
        self.data_path = path.join(root_path,user_name,split)
#         print("datapath:",path.abspath(self.data_path))
        
        self.split = split
        self.finger_type_list = finger_type_list
        self.classify_num = classify_num
        self.inference_mode = inference_mode
        
        self.files = glob(path.join(self.data_path,'*'))
        
        self.emg_data_list = []
        self.leap_data_list = []
        
        for file_path in self.files:
            emg_data = pd.read_csv(path.join(file_path,"emg_filtered.csv"),header=None).to_numpy()
            
            leap_data = json.load(open(path.join(file_path,"leap_data_processed.json"),mode='r', encoding='utf-8'))
            leap_data_temp = data_utils.parse_leap_all_finger(leap_data,self.finger_type_list) # [1000,N,4]
            leap_data = leap_data_temp.reshape(len(leap_data_temp),-1) # [1000,N*4]
            
            self.emg_data_list.append(emg_data)
            self.leap_data_list.append(leap_data)
        
        # emg normalize z-score 
        self.emg_data_list = data_utils.emg_z_norm(self.emg_data_list)
        # smooth leap motion data
        self.leap_data_list = data_utils.smooth_gt(self.leap_data_list,self.finger_type_list)
        
        if self.inference_mode == False:
            # 将gt归一化到[0,1]，记录每2s窗口的gt的最大值、最小值
            min_vals,max_vals,_ = data_utils.gt_normalize(self.leap_data_list,finger_type_list) # [420,4] [420,4]
            # classify each min, max value to each bin
            min_bin_lower_limit,min_bin_upper_limit,min_gt_regression = data_utils.bin_limit(min_vals,self.finger_type_list,self.classify_num)
            max_bin_lower_limit,max_bin_upper_limit,max_gt_regression = data_utils.bin_limit(max_vals,self.finger_type_list,self.classify_num)

            self.gts_regression = np.concatenate((min_gt_regression, max_gt_regression), axis=1)
            self.bin_lower_limits = np.concatenate((min_bin_lower_limit, max_bin_lower_limit), axis=1)
            self.bin_upper_limits = np.concatenate((min_bin_upper_limit, max_bin_upper_limit), axis=1)
        
        elif self.inference_mode == True:
            self.bin_indexs = classification_cascade_inference.get_cascade_inference(root_path,output_path,user_name,classify_num)
        
    def __len__(self):
        return len(self.files)
    
    def get_limit(self,tags):
        default_min_vals_list = []
        default_max_vals_list = []

        for finger_type in self.finger_type_list:
            min_vals,max_vals = data_utils.default_min_max(finger_type)
            default_min_vals_list += min_vals
            default_max_vals_list += max_vals

        default_min_vals_list += default_min_vals_list
        default_max_vals_list += default_max_vals_list
        
        angle_template = [[] for _ in range(len(default_min_vals_list))]
        for idx in range(len(angle_template)):
            interval = (default_max_vals_list[idx] - default_min_vals_list[idx]) / self.classify_num
            angle_template[idx] = [[default_min_vals_list[idx] + i*interval, default_min_vals_list[idx] + (i+1)*interval] for i in range(self.classify_num)]

        upper_limit = []
        lower_limit = []
        
        for i,tag in enumerate(tags):
            for idx in range(len(angle_template[i])):
                if tag == idx:
                    upper_limit.append(angle_template[i][idx][1])
                    lower_limit.append(angle_template[i][idx][0])
                    
                    
        return np.array(lower_limit),np.array(upper_limit)
    
    def __getitem__(self, idx):
        # emg_data[idx]: 1000*2  min_data[idx]:2*len(finger_type) max_data[idx]:2*len(finger_type)
        emg_data = self.emg_data_list[idx]
        
        emg = emg_data.astype(np.float32)
        
        if self.inference_mode == False:
            gt_regression = self.gts_regression[idx]
            bin_lower_limit = self.bin_lower_limits[idx]
            bin_upper_limit = self.bin_upper_limits[idx]

            gt_regression = gt_regression.astype(np.float32)
            bin_lower_limit = bin_lower_limit.astype(np.float32)
            bin_upper_limit = bin_upper_limit.astype(np.float32)

            return emg,gt_regression,bin_lower_limit,bin_upper_limit
        
        elif self.inference_mode == True:
            bin_index = self.bin_indexs[idx]
            bin_lower_limit,bin_upper_limit = self.get_limit(bin_index)
            
            return emg,bin_index,bin_lower_limit,bin_upper_limit
    
    def get_test(self, idx):
        return self.__getitem__(idx)